In [ ]:
from collections import OrderedDict
import itertools
import os

import matplotlib.pyplot as plt
import numpy as np
import open3d as o3d
import open3d.ml.torch as ml3d
import torch
import torch.utils.dlpack

In [ ]:
def measure_time(fn, min_samples=10, max_samples=100, max_time_in_sec=100.0):
    """Measure time to run fn. Returns the elapsed time each run."""
    from time import perf_counter_ns
    t = []
    for i in range(max_samples):
        if sum(t) / 1e9 >= max_time_in_sec and i >= min_samples:
            break
        t.append(-perf_counter_ns())
        try:
            ans = fn()
        except Exception as e:
            print(e)
            return np.array([np.nan])
        t[-1] += perf_counter_ns()
        del ans
    print('.', end='')
    return np.array(t) / 1e9

In [ ]:
def generate_datasets(search_type="radius", sort=None):
    # setup dataset examples
    datasets = OrderedDict()

    if search_type != "knn":
        basedir = "../../test_data"

        # real data
        # skip small_tower dataset for hybrid search and sorted fixed radius search due to the memory limit
        if search_type == "radius" and not sort:
            pcd = o3d.t.io.read_point_cloud(
                os.path.join(basedir, 'small_tower.ply'))
            points = queries = pcd.point['points']
            datasets['sfm'] = {'points': points, 'queries': queries}

        pcd_ds = o3d.t.io.read_point_cloud(os.path.join(basedir, 'canyon.ply'))

        points = pcd_ds.point['points'].to(o3d.core.Dtype.Float32)
        pcd_qs = o3d.t.io.read_point_cloud(
            os.path.join(basedir, 'fluid_1000.ply'))
        queries = pcd_qs.point['points'].to(o3d.core.Dtype.Float32)
        datasets['fluid'] = {'points': points, 'queries': queries}

        # random data
        for i, ds_qs in enumerate(
                itertools.product((1000, 10000, 100000), repeat=2)):
            points = o3d.core.Tensor.from_numpy(
                np.random.rand(ds_qs[0], 3).astype(np.float32))
            queries = o3d.core.Tensor.from_numpy(
                np.random.rand(ds_qs[1], 3).astype(np.float32))
            datasets['random{}'.format(i)] = {
                'points': points,
                'queries': queries
            }
    else:
        # random data
        for i, ds_knn in enumerate(
                itertools.product((1000, 10000, 100000, 1000000),
                                  (3, 16, 32, 64))):
            points = o3d.core.Tensor.from_numpy(
                np.random.rand(ds_knn[0], ds_knn[1]).astype(np.float32))
            queries = o3d.core.Tensor.from_numpy(
                np.random.rand(1000, ds_knn[1]).astype(np.float32))
            datasets['random{}'.format(i)] = {
                'points': points,
                'queries': queries
            }
    return datasets

In [ ]:
class O3D:

    def __init__(self, device_str, search_type, sort=False):
        self.device_str = device_str
        self.device = o3d.core.Device(
            o3d.core.Device.CUDA,
            0) if device_str == "gpu" else o3d.core.Device(
                o3d.core.Device.CPU, 0)
        self.search_type = search_type
        self.sort = sort

    def prepare_input(self, dataset, query):
        self.dataset = dataset.to(self.device)
        self.query = query.to(self.device)

    def setup(self, radius=None):
        dataset, search_type = self.dataset, self.search_type

        nns = o3d.core.nns.NearestNeighborSearch(dataset)
        status = False

        if search_type == "knn":
            status = nns.knn_index()
        elif search_type == "radius":
            status = nns.fixed_radius_index(radius)
        elif search_type == "hybrid":
            status = nns.hybrid_index(radius)

        if not status:
            raise Exception("index failed")

        self.nns = nns
        return self.nns

    def search(self, knn=None, radius=None):
        nns, query, search_type = self.nns, self.query, self.search_type

        ans = None
        if search_type == "knn":
            ans = nns.knn_search(query, knn)
        elif search_type == "radius":
            ans = nns.fixed_radius_search(query, radius, self.sort)
        elif search_type == "hybrid":
            ans = nns.hybrid_search(query, radius, knn)

        return ans

    @property
    def name(self):
        return f"o3d_{self.search_type}_{self.device_str}_sort" if self.sort else f"o3d_{self.search_type}_{self.device_str}"


class Torch:

    def __init__(self, device_str, search_type):
        assert search_type == "radius", "Torch only support fixed radius search."
        self.device_str = device_str
        self.device = torch.device("cuda" if device_str == "gpu" else "cpu")
        self.cuda_sync = device_str == "gpu"

    def prepare_input(self, dataset, query):
        self.dataset = torch.utils.dlpack.from_dlpack(dataset.to_dlpack()).to(
            self.device)
        self.query = torch.utils.dlpack.from_dlpack(query.to_dlpack()).to(
            self.device)
        if self.cuda_sync:
            torch.cuda.synchronize()

    def setup(self, radius):
        dataset = self.dataset

        self.table = ml3d.ops.build_spatial_hash_table(
            dataset,
            radius,
            torch.LongTensor([0, dataset.shape[0]]),
            1 / 32,
        )

        if self.cuda_sync:
            torch.cuda.synchronize()

        return self.table

    def search(self, knn=None, radius=None):
        table, dataset, query = self.table, self.dataset, self.query

        ans = ml3d.ops.fixed_radius_search(
            dataset,
            query,
            radius,
            torch.LongTensor([0, dataset.shape[0]]),
            torch.LongTensor([0, query.shape[0]]),
            **table._asdict(),
            return_distances=True)
        if self.cuda_sync:
            torch.cuda.synchronize()
        return ans

    @property
    def name(self):
        return f"torch_radius_{self.device_str}"

In [ ]:
def run_benchmark(datasets, methods, neighbors):
    # collects runtimes for all examples
    results = OrderedDict()

    def compute_avg_radii(points, queries, neighbors):
        """Computes the radii based on the number of neighbors"""
        from scipy.spatial import cKDTree
        tree = cKDTree(points.numpy())
        avg_radii = []
        for k in neighbors:
            dist, _ = tree.query(queries.numpy(), k=k)
            avg_radii.append(np.mean(dist.max(axis=-1)))
        return avg_radii

    for example_name, example in datasets.items():
        dataset = example["points"]
        query = example["queries"]
        radii = compute_avg_radii(dataset, query, neighbors)

        for nn, radius in zip(neighbors, radii):
            print(
                f"\nname: {example_name}, dataset: {dataset.shape}, query: {query.shape}, nn: {nn}"
            )
            example_results = OrderedDict({
                'dataset_size': dataset.shape[0],
                'query_size': query.shape[0],
                'dim': dataset.shape[1],
                'neighbors': nn
            })

            for m in methods:
                empty_o3d, empty_torch = False, False
                if isinstance(m, O3D) and m.device_str == "gpu":
                    empty_o3d = True
                elif isinstance(m, Torch) and m.device_str == "gpu":
                    empty_torch = True

                # prepare inputs
                m.prepare_input(dataset, query)

                # measure setup time
                t_setup = measure_time(lambda: m.setup(radius))
                example_results[f'{m.name}_setup'] = t_setup

                # measure search time
                t_search = measure_time(lambda: m.search(knn=nn, radius=radius))
                example_results[f'{m.name}_search'] = t_search

                # calculate total time
                n = min(t_setup.shape[0], t_search.shape[0])
                t_total = t_setup[:n] + t_search[:n]
                example_results[f'{m.name}_total'] = t_total

                # delete data and clear cache
                del m
                if empty_o3d:
                    with o3d.utility.VerbosityContextManager(
                            o3d.utility.VerbosityLevel.Warning):
                        o3d.core.cuda.release_cache()
                if empty_torch:
                    torch.cuda.empty_cache()

            results[f'{example_name} {nn}'] = example_results
    return results

In [ ]:
def generate_plots(results, method_names):
    num_methods = len(method_names)
    labels = [f'{m}_setup' for m in method_names] + [
        f'{m}_search' for m in method_names
    ] + [f'{m}_total' for m in method_names]

    fig, axes = plt.subplots(len(results), 3, figsize=(32, 8 * len(results)))
    for i, x in enumerate(results):
        values = results[x]
        row_label = '{} (ds={}, qs={}, dim={}, k={})'.format(
            x.split()[0], values['dataset_size'], values['query_size'],
            values['dim'], values['neighbors'])
        for j in range(3):
            ax = axes[i, j]
            data_labels = labels[j * num_methods:(j + 1) * num_methods]
            axis_labels = [
                "_".join(label.split("_")[:-1]) for label in data_labels
            ]
            data = [results[x][l] for l in data_labels]
            ax.boxplot(data, showfliers=False, labels=axis_labels)
            ax.set_ylabel('sec')
            if j == 0:
                ax.set_title("Setup time")
            elif j == 1:
                ax.set_title(f'{row_label}\n\nSearch time')
            elif j == 2:
                ax.set_title("Total time")

## Fixed Radius Search Benchmark

Methods
- Open3D CPU
- Open3D GPU
- Torch CPU
- Torch GPU

In [ ]:
methods = [
    O3D("cpu", "radius"),
    Torch("cpu", "radius"),
    O3D("gpu", "radius"),
    Torch("gpu", "radius")
]
method_names = [m.name for m in methods]

# generate dataset
datasets = generate_datasets(search_type="radius")

# run benchmark
results = run_benchmark(datasets, methods, (2, 20, 35))

In [ ]:
generate_plots(results, method_names)

## Hybrid Search Benchmark
Methods
- Open3D CPU
- Open3D GPU

In [ ]:
methods = [O3D("cpu", "hybrid"), O3D("gpu", "hybrid")]
method_names = [m.name for m in methods]

# generate dataset
datasets = generate_datasets(search_type="hybrid")

# run benchmark
results = run_benchmark(datasets, methods, (2, 20, 35))
generate_plots(results, method_names)

## Knn Search Benchmark

Methods
- Open3D CPU
- Open3D GPU


In [ ]:
methods = [O3D("cpu", "knn"), O3D("gpu", "knn")]
method_names = [m.name for m in methods]

# generate dataset
datasets = generate_datasets(search_type="knn")

# run benchmark
results = run_benchmark(datasets, methods, (1, 8, 16, 32))

In [ ]:
generate_plots(results, method_names)